# Table Building

## Setup

In [1]:
import numpy as np
import pandas as pd
import re
import json

## Read Data

In [2]:
with open('../data/cleaned_data.json') as f:
    data = json.load(f)
f.close()

## Build `LIB` table

### Initialize dictionary we'll use to make the data frame later

In [3]:
lib_dict = dict()

### Loop over json and add songs and metadata to `lib_dict` with an arbitrary song ID

In [28]:
idx = 0
# Loop over artists
for artist in data.keys():
    # Loop over albums
    for album in data[artist]['Albums'].keys():
        # Loop over tracks
        for track in data[artist]['Albums'][album]['tracklist'].keys():
            lib_dict[idx] = {
                'title':track,
                'json_path':(f'data[{artist}][\'Albums\'][{album}][\'tracklist\'][track]'),
                'artist':artist,
                'album':album,
                'genres':data[artist]['Metadata']['genres'],
                'release_date':data[artist]['Albums'][album]['release_date'],
                'label':data[artist]['Albums'][album]['label'],
                'track_number':data[artist]['Albums'][album]['tracklist'][track]['track_number'],
                'duration_ms':data[artist]['Albums'][album]['tracklist'][track]['duration_ms'],
                'track_number':data[artist]['Albums'][album]['tracklist'][track]['track_number'],
                'audio_information':data[artist]['Albums'][album]['tracklist'][track]['audio_information'],
                'lyrics':data[artist]['Albums'][album]['tracklist'][track]['lyrics']
            }
            idx += 1

### Convert `lib_dict` to data frame

In [30]:
lib = pd.DataFrame.from_dict(lib_dict).T
lib.index.name = 'song_id'

In [31]:
lib.head()

,title,json_path,artist,album,genres,release_date,label,track_number,duration_ms,audio_information,lyrics
song_id,,,,,,,,,,,
0,Pay Your Way In Pain,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,1,183880,"{'danceability': 0.677, 'energy': 0.826, 'key'...",\n(Ow)\nOh-oh-oh\n\nYou got to pay your way in...
1,Down And Out Downtown,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,2,222240,"{'danceability': 0.61, 'energy': 0.502, 'key':...",\nLast night's heels\nOn the mornin' train\nIt...
2,Daddy's Home,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,3,199613,"{'danceability': 0.774, 'energy': 0.372, 'key'...",\nI signed autographs in the visitation room\n...
3,Live In The Dream,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,4,389546,"{'danceability': 0.426, 'energy': 0.339, 'key'...",\nHello\nDo you know where you are?\nYou've be...
4,The Melting Of The Sun,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,5,257519,"{'danceability': 0.52, 'energy': 0.463, 'key':...","\nSo sorry, missed the party\nHello, on the da..."


### Expand `audio_information` to individual columns

In [32]:
audio_info = lib.audio_information

In [37]:
audio_frame = audio_info.apply(pd.Series)

In [39]:
audio_frame.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
song_id,,,,,,,,,,,,,,,,,,
0,0.677,0.826,9,-6.800,1,0.3680,0.3950,0.00000,0.5710,0.4980,124.821,audio_features,1nLaoWVhNRZ38irY0LZjL6,spotify:track:1nLaoWVhNRZ38irY0LZjL6,https://api.spotify.com/v1/tracks/1nLaoWVhNRZ3...,https://api.spotify.com/v1/audio-analysis/1nLa...,183880,4
1,0.610,0.502,6,-8.409,1,0.0820,0.0578,0.01410,0.1660,0.6730,151.906,audio_features,4htB9ArZCpquXlXnkKjAgk,spotify:track:4htB9ArZCpquXlXnkKjAgk,https://api.spotify.com/v1/tracks/4htB9ArZCpqu...,https://api.spotify.com/v1/audio-analysis/4htB...,222240,4
2,0.774,0.372,11,-9.796,0,0.0956,0.6150,0.00148,0.1060,0.6020,110.036,audio_features,6DeE4f5DFeWycZz8f6UEht,spotify:track:6DeE4f5DFeWycZz8f6UEht,https://api.spotify.com/v1/tracks/6DeE4f5DFeWy...,https://api.spotify.com/v1/audio-analysis/6DeE...,199613,4
3,0.426,0.339,2,-10.262,1,0.0270,0.7510,0.06450,0.0871,0.0689,80.224,audio_features,27JieQ9wlcYMf1SwTczPUl,spotify:track:27JieQ9wlcYMf1SwTczPUl,https://api.spotify.com/v1/tracks/27JieQ9wlcYM...,https://api.spotify.com/v1/audio-analysis/27Ji...,389547,4
4,0.520,0.463,10,-10.429,1,0.0958,0.6090,0.00140,0.4260,0.4690,159.931,audio_features,3WD3w5uSzAGJWrNFnHhi1a,spotify:track:3WD3w5uSzAGJWrNFnHhi1a,https://api.spotify.com/v1/tracks/3WD3w5uSzAGJ...,https://api.spotify.com/v1/audio-analysis/3WD3...,257520,4


#### Drop some redundant or unnecessary columns

In [40]:
audio_frame.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [41]:
audio_frame = audio_frame.drop(['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms'], axis=1)

In [42]:
audio_frame.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
song_id,,,,,,,,,,,,
0,0.677,0.826,9,-6.800,1,0.3680,0.3950,0.00000,0.5710,0.4980,124.821,4
1,0.610,0.502,6,-8.409,1,0.0820,0.0578,0.01410,0.1660,0.6730,151.906,4
2,0.774,0.372,11,-9.796,0,0.0956,0.6150,0.00148,0.1060,0.6020,110.036,4
3,0.426,0.339,2,-10.262,1,0.0270,0.7510,0.06450,0.0871,0.0689,80.224,4
4,0.520,0.463,10,-10.429,1,0.0958,0.6090,0.00140,0.4260,0.4690,159.931,4


#### Add `audio_frame` to `lib`

In [43]:
lib = lib.join(audio_frame)

In [45]:
lib.head()

,title,json_path,artist,album,genres,release_date,label,track_number,duration_ms,audio_information,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
song_id,,,,,,,,,,,,,,,,,,,,,
0,Pay Your Way In Pain,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,1,183880,"{'danceability': 0.677, 'energy': 0.826, 'key'...",...,9,-6.800,1,0.3680,0.3950,0.00000,0.5710,0.4980,124.821,4
1,Down And Out Downtown,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,2,222240,"{'danceability': 0.61, 'energy': 0.502, 'key':...",...,6,-8.409,1,0.0820,0.0578,0.01410,0.1660,0.6730,151.906,4
2,Daddy's Home,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,3,199613,"{'danceability': 0.774, 'energy': 0.372, 'key'...",...,11,-9.796,0,0.0956,0.6150,0.00148,0.1060,0.6020,110.036,4
3,Live In The Dream,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,4,389546,"{'danceability': 0.426, 'energy': 0.339, 'key'...",...,2,-10.262,1,0.0270,0.7510,0.06450,0.0871,0.0689,80.224,4
4,The Melting Of The Sun,data[St. Vincent]['Albums'][Daddy's Home]['tra...,St. Vincent,Daddy's Home,"[art pop, etherpop, indie rock, indietronica, ...",2021-05-14,Loma Vista Recordings,5,257519,"{'danceability': 0.52, 'energy': 0.463, 'key':...",...,10,-10.429,1,0.0958,0.6090,0.00140,0.4260,0.4690,159.931,4


### Change `dtypes` to reasonable types

In [47]:
lib.dtypes

title                 object
json_path             object
artist                object
album                 object
genres                object
release_date          object
label                 object
track_number          object
duration_ms           object
audio_information     object
lyrics                object
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
time_signature         int64
dtype: object